# Analysis for first sky scans from Alto Chorillos
JCH - June 2023


In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,10))
rc('font',size=12)

from scipy.signal import medfilt
from scipy.interpolate import interp1d
import glob
import numpy as np

#### Resampling
from scipy.ndimage import gaussian_filter1d
from scipy.signal import resample
from scipy.signal import resample_poly


from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft

from importlib import reload
import healpy as hp
from datetime import datetime

from astropy.visualization import astropy_mpl_style, quantity_support
# plt.style.use(astropy_mpl_style)
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun, get_body

import pysm3
import pysm3.units as u
from pysm3 import utils


import sys
reps = ['/Users/hamilton/Python/GitQubic/qubic/qubic/scripts/DiversJC/CalibSalta/']
for rep in reps:     
    if rep not in sys.path:
        sys.path.append(rep)
import fitting as fit
import time_domain_tools as tdt


### Files (from Giuseppe D'Alessandro): 
- 2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night
- 2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day
- 2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day
- 2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened
- 2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened
- 2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime

### Copy them: 
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-28/2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-28/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-28/2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-28/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-27/2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-27/

In [ ]:
# DirData = '/Volumes/HD JCH SSD/Qubic/QubicData/CommissioningTD/'
DirData = '/Users/hamilton/Qubic/Data/CommissioningTD/'

files = [
    '2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night', 
    '2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day',     
    '2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day',   
    '2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened',      
    '2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened',      
    '2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime'
]

### Read one file
file = files[3]
day = file[:10]
print("Reading:")
print(day, file)

thedata = DirData + '/' + day +'/' + file
### Read data
a = qubicfp()
a.read_qubicstudio_dataset(thedata)

tt_init, alltod_init = a.tod()
### We directly resample to 10Hz ########################################
### Current sampling
deltat = np.max(tt_init)-np.min(tt_init)
ns = len(tt_init)
fs = ns/deltat

### Target sampling
newfs_approx = 10.
new_ns = int(2 * ns * newfs_approx / fs)
newfs = new_ns / deltat
alltod, tt = resample(alltod_init, new_ns, t=tt_init, axis=1)
##########################################################################


thk = a.timeaxis(datatype='platform')
az = a.azimuth()
el = a.elevation()

Tbath = a.Tbath
del(a)

### We remove tt[0]
tinit = tt[0]
tt -= tinit
thk -= tinit


In [ ]:

rc('figure',figsize=(20,10))
rc('font',size=12)

TESNum = 96
mytod = alltod[TESNum-1,:]


new_fsample = 10.
new_nsample = np.max(tt)-np.min(tt)

ps, ff = ft.power_spectrum(tt, mytod)


subplot(2,1,1)
plot(tt, mytod)
xlabel('Time [sec]')
ylabel('TOD')
title('TES #{}'.format(TESNum))

subplot(2,2,3)
plot(ff, ps)
plot(ff, gaussian_filter1d(ps, 10))
yscale('log')
xscale('log')
xlabel('Freq [Hz]')
ylabel('PSD')
title('TES #{}'.format(TESNum))

subplot(2,2,4)
plot(ff, ps)
plot(ff, gaussian_filter1d(ps, 30))
xlim(0, 10)
yscale('log')
xlabel('Freq [Hz]')
ylabel('PSD')
title('TES #{}'.format(TESNum))
for i in range(10):
    axvline(x=i, alpha=0.1)
grid()


tight_layout()

In [ ]:
ok = el > 30
az = az[ok]
el = el[ok]
thk = thk[ok]

rc('figure',figsize=(20,12))
rc('font',size=12)

TESnum = 33
tod = alltod[TESnum-1,:]

subplot(2,2,1)
plot(tt, tod, label='TES #{}'.format(TESnum))
xlabel('t')
ylabel('TOD')

subplot(2,2,2)
plot(Tbath[0]-tinit, Tbath[1], label='TBath')
xlabel('t')
ylabel('TBath')

subplot(2,3,4)
plot(az, el)
xlabel('az')
ylabel('el')

subplot(2,3,5)
plot(thk, az)
xlabel('t')
ylabel('az')

subplot(2,3,6)
plot(thk, el)
xlabel('t')
ylabel('el')


In [ ]:
reload(tdt)
### 0: Identify scan types and numbers
speedmin = 0.035
scantype_hk, azt, elt, scantype, vmean = tdt.identify_scans(thk, az, el, tt=tt, doplot=True, plotrange=[0,2000], thr_speedmin=speedmin)
print('Mean Velocity: {0:5.3f} deg/s'.format(vmean))
daz = max(azt)-min(azt)
print('Delta_az = {}'.format(daz))
fsignal = vmean/daz*np.cos(np.radians(np.mean(el)))
print('Signal location in frequency: {} Hz'.format(fsignal))


### Create a QubicSampling object from these
azqubic = 168.5
AltoChorrillos = EarthLocation(lat=-24.18629364167771*u.deg, lon=-66.478206557511*u.deg, height=4890*u.m)
from qubic.samplings import QubicSampling
date_obs = str(datetime.utcfromtimestamp(tinit))
print('Observations started at: {} UTC'.format(date_obs))

qs = QubicSampling(azimuth=azt + azqubic, elevation=elt, time=tt+tinit,
                   date_obs=date_obs, longitude=float(AltoChorrillos.lon/u.deg), latitude = float(AltoChorrillos.lat/u.deg))

ra, dec = qs.equatorial.T
ll, bb = qs.galactic.T

figure()
subplot(2,3,1)
plot(qs.time, qs.equatorial[:,0])
xlabel('UTC Time from QS (sec)')
ylabel('RA from QS (deg)')
subplot(2,3,2)
plot(qs.time , qs.equatorial[:,1])
xlabel('UTC Time from QS (sec)')
ylabel('DEC from QS (deg)')
subplot(2,3,3)
plot(qs.equatorial[:,0], qs.equatorial[:,1])
xlabel('RA from QS (deg)')
ylabel('DEC from QS (deg)')
tight_layout()

subplot(2,3,4)
plot(qs.azimuth, qs.elevation, label='From QS')
plot(az + azqubic, el, '--', label='From HK')
xlabel('az (deg)')
ylabel('el (deg)')

subplot(2,3,5)
plot(qs.time, qs.azimuth, label='From QS')
plot(thk, az + azqubic, '--', label='From HK')
xlabel('time (sec)')
ylabel('az (deg)')

subplot(2,3,6)
plot(qs.time, qs.elevation, label='From QS')
plot(thk, el, '--', label='From HK')
xlabel('time (sec)')
ylabel('el (deg)')

fig = figure()

fig.suptitle(file, fontsize=14, y=0.75)


ax = fig.add_subplot(1,2,1,projection='mollweide')
grid()
title('Equatorial')

phi = np.radians(((qs.equatorial[:,0]+180) % 360)-180)
theta = np.radians(qs.equatorial[:,1])

plot(phi, theta ,'ro')
xlabel('RA from QS (deg)')
ylabel('DEC from QS (deg)')


ax = fig.add_subplot(1,2,2,projection='mollweide')
grid()
title('Galactic')

phi = np.radians(((qs.galactic[:,0]+180) % 360)-180)
theta = np.radians(qs.galactic[:,1])

plot(phi, theta ,'ro')
xlabel('RA from QS (deg)')
ylabel('DEC from QS (deg)')
tight_layout()



In [ ]:
# What does PySM say ?
import pysm3
import pysm3.units as u
from pysm3 import utils

nspysm = 256
#### Build "simple" simulateed TOD (assuming single peak)
sky=pysm3.Sky(nside=nspysm, preset_strings=['d0'], output_unit="uK_CMB")
fakesky=np.array(sky.get_emission(150 * u.GHz, None).T * utils.bandpass_unit_conversion(150*u.GHz, None, u.uK_CMB))[:,0]
fakesky = hp.smoothing(fakesky, fwhm=np.radians(1))

hp.mollview(fakesky, sub=(1,2,1))

ip = hp.ang2pix(nspysm, np.radians(90-bb), np.radians(ll))
simtod = fakesky[ip]

psfake, fffake = ft.power_spectrum(tt, simtod)

subplot(1,2,2)
plot(fffake, psfake, label='Initial', alpha=0.5)
axvline(x=fsignal, ls=':', label='Expected signal location')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')
legend()


In [ ]:
# What does PySM say ?
import qubic
from qubicpack.utilities import Qubic_DataDir
import gc
import qubicpack as qp
# Function to go from QubicSoft (Sims) indices (0-247) to QubicPack (data) indices (0-255)
### The 8 thermometers are not in QubicSoft

def iQS2iQP(indexQS):
    qpnumi, qpasici = qp.pix2tes.pix2tes(indexQS+1)
    return qpnumi+(qpasici-1)*128-1

def iQP2iQS(indexQP):
    QStesnum = qp.pix2tes.tes2pix(indexQP%128+1, indexQP//128+1)
    return QStesnum-1


global_dir = Qubic_DataDir()
dictfilename = global_dir + '/dicts/pipeline_MoonSalta.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside'] = 256
d['config'] = 'TD'
d['kind'] = 'I'
d['nf_sub'] = 5
d['nf_recon'] = 2
d['MultiBand'] = True
d['sweeping_pointing_deadtime'] = False
d['noiseless'] = True
d['detector_tau'] = 0.01

s = qubic.QubicScene(d)
q = qubic.QubicInstrument(d)

y0 = fakesky.copy()

### We use our fake moon-frame pointing qsmoon
qs.fix_az = False
a = qubic.QubicAcquisition(q, qs, s, d)

TOD_sim, mapconv = a.get_observation(y0, noiseless=d['noiseless'], convolution = True)
del a
gc.collect()   ### Important ! Frees the meory

subplot(2,1,1)
plot(TOD_sim[0,:])

psfake, fffake = ft.power_spectrum(tt, TOD_sim[0,:])

subplot(2,1,2)
plot(fffake, psfake, label='Initial', alpha=0.5)
axvline(x=fsignal, ls=':', label='Expected signal location')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')
legend()


In [ ]:
def healpix_map(azt, elt, tod, flags=None, flaglimit=0, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    if flags is None:
        flags = np.zeros(len(azt))
    
    ok = flags <= flaglimit 
    return healpix_map_(azt[ok], elt[ok], tod[ok], nside=nside, countcut=countcut, unseen_val=unseen_val)


def healpix_map_(azt, elt, tod, nside=128, countcut=0, unseen_val=hp.UNSEEN):
    ips = hp.ang2pix(nside, azt, elt, lonlat=True)
    mymap = np.zeros(12*nside**2)
    mapcount = np.zeros(12*nside**2)
    for i in range(len(azt)):
        mymap[ips[i]] += tod[i]
        mapcount[ips[i]] += 1
    unseen = mapcount <= countcut
    mymap[unseen] = unseen_val
    mapcount[unseen] = unseen_val
    mymap[~unseen] = mymap[~unseen] / mapcount[~unseen]
    return mymap, mapcount

def display_one(mapsb, anatype='', sub=(1,1,1), nlo=3, nhi=3, reso=12, rot=[0,50]):
    unseen = (mapsb == hp.UNSEEN)
    mm, ss = ft.meancut(mapsb[~unseen], 3)
    hp.gnomview(mapsb, rot=rot, reso=reso, sub=sub, title=anatype+'\n Both scans $\sigma$ = {0:5.3g}'.format(ss), min=-nlo*ss, max=nhi*ss)


def do_display_all(mapsb, mapsb_pos, mapsb_neg, mapav, mapdiff, mapdiff2, rot=[0,50], anatype='', reso=12, myrange=None, TESNum = None, graticule=True):
    unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)
    mm, ss = ft.meancut(mapsb[~unseen], 3)
    
    if myrange is None:
        mini = -3*ss
        maxi = 3*ss
    else:
        mini = myrange[0]
        maxi = myrange[1]
        
    if TESNum != None:
        anatype += '\n TES# {}'.format(TESNum)

    figure()
    hp.gnomview(mapsb, rot=rot, reso=reso, sub=(2,3,1), title=anatype+'\n Both scans $\sigma$ = {0:5.4g}'.format(ss), min=mini, max=maxi)
    if graticule:
        hp.graticule()
    mmp, ssp = ft.meancut(mapsb_pos[~unseen], 3)
    hp.gnomview(mapsb_pos, rot=rot, reso=reso, sub=(2,3,2), title=anatype+'\n Pos scans $\sigma$ = {0:5.4g}'.format(ssp), min=mini, max=maxi)
    if graticule:
        hp.graticule()
    mmn, ssn = ft.meancut(mapsb_neg[~unseen], 3)
    hp.gnomview(mapsb_neg, rot=rot, reso=reso, sub=(2,3,3), title=anatype+'\n Neg scans $\sigma$ = {0:5.4g}'.format(ssn), min=mini, max=maxi)
    if graticule:
        hp.graticule()
    mma, ssa = ft.meancut(mapav[~unseen], 3)
    hp.gnomview(mapav, rot=rot, reso=reso, sub=(2,3,4), title=anatype+'\n Av of Both scans $\sigma$ = {0:5.4g}'.format(ssa), min=mini, max=maxi)
    if graticule:
        hp.graticule()
    mmd, ssd = ft.meancut(mapdiff[~unseen], 3)
    hp.gnomview(mapdiff, rot=rot, reso=reso, sub=(2,3,5), title=anatype+'\n Diff/2 of both scans $\sigma$ = {0:5.4g}'.format(ssd), min=mini, max=maxi)
    if graticule:
        hp.graticule()
    mmd2, ssd2 = ft.meancut(mapdiff2[~unseen], 3)
    hp.gnomview(mapdiff2, rot=rot, reso=reso, sub=(2,3,6), title=anatype+'\n Both - Av $\sigma$ = {0:5.4g}'.format(ssd2), min=mini/ss**ssd, max=maxi/ss*ssd)
    if graticule:
        hp.graticule()
    

def display_all(mapsb, mapsb_pos, mapsb_neg, anatype='', rot=[0,50], highcontrast=False, reso=12, myrange=None, TESNum=None):
    unseen = (mapsb == hp.UNSEEN) | (mapsb_pos == hp.UNSEEN) | (mapsb_neg == hp.UNSEEN)

    ### Average of back and Forth
    mapav = (mapsb_pos + mapsb_neg)/2
    mapav[unseen] = hp.UNSEEN

    ### Difference of back and Forth
    mapdiff = (mapsb_pos - mapsb_neg) / 2
    mapdiff[unseen] = hp.UNSEEN

    ### Difference of All and Av
    mapdiff2 = (mapav - mapsb)
    mapdiff2[unseen] = hp.UNSEEN
    
    if highcontrast:
        myrange = [-np.max(mapsb[~unseen])/10, np.max(mapsb[~unseen])*0.8]
        
    do_display_all(mapsb, mapsb_pos, mapsb_neg, mapav, mapdiff, mapdiff2, rot=rot, anatype=anatype, reso=reso, myrange=myrange, TESNum=TESNum)
    
    

## Level 0 treatment & Coaddition in local coordinates

In [ ]:
reload(tdt)
reload(ft)
def mypipe(mytod, tt, azt, elt, scantype, doplot=False, 
           nsplines_global=40,
           nbspl_scan=5,
           nbins_scan=30, lowcut=None, highcut=None, order=5, verbose=False, offsets=True):
    
    # 1. High-Pass or Remove a slow spline to the data
    if highcut or lowcut:
        mytod = ft.filter_data(tt, mytod, lowcut = lowcut, highcut = highcut, rebin=False, verbose=verbose, notch=None, order=order)
    else:
        mytod = tdt.remove_drifts_spline(tt, mytod, nsplines=nsplines_global, doplot=doplot)

    # 2. Offset removal scan by scan using median 
    #    (here we just want to have all scans at the same level before decorrelating from azimuth)
    mytod = -mytod
    if offsets:
        mytod = tdt.remove_offset_scan(mytod, scantype, method='median')

    # 3. Remove azimuth correlation
    # low degree polynomials or splines...
    if nbins_scan !=0:
        mytod = tdt.decorel_azel(mytod, azt, elt, scantype, doplot=doplot, nbins=nbins_scan, n_el=np.max(np.abs(scantype)), 
                                 nbspl=nbspl_scan)

    # 4. remove offsets again but this time with mode method as it appears to be less affected 
    #    by the presence of the peaks (no underestimation of the offset resultingg is shadow around the peaks)
    #mytod = tdt.remove_offset_scan(mytod, scantype, method='mode')
    if offsets:
        mytod = tdt.remove_offset_scan(mytod, scantype, method='median')
    
    return mytod



    

# From Michel: NET
96: ~ 30mK.sqrt(s)



In [ ]:
rc('figure',figsize=(16,10))
rc('font',size=12)

# Filtering params
thr_seconds = 16.
counts_thr = thr_seconds / (tt[1]-tt[0]) * 0 + 1

nsplines_global = 100
lowcut = 0.0038
highcut = 0.10
nbspl_scan = 10
nbins_scan = 0#50
order = 5
nsig = 3
offsets=False


reload(tdt)
# Map-making
anatype = ''

TESNum = 96#3#133#95#33

tod = alltod[TESNum-1,:]

ps, ff = ft.power_spectrum(tt, tod)


mytod = mypipe(tod, tt, azt, elt, scantype, doplot=True, verbose=True,
               nsplines_global=nsplines_global,
               lowcut = lowcut,
               highcut = highcut,
               nbspl_scan=nbspl_scan,
               nbins_scan=nbins_scan,
               order=order, offsets=offsets)

ps2, ff2 = ft.power_spectrum(tt, mytod)

figure()
plot(ff, ps, label='Initial', alpha=0.5)
plot(ff2, ps2, label='Final', alpha=0.5)
axvline(x=fsignal, ls=':', label='Expected signal location')
axvline(x=lowcut, ls='--', lw=2, color='g', label='High Pass')
axvline(x=highcut, ls=':', lw=2, color='g', label='Low Pass')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')
title('Data TES #{}'.format(TESNum))
legend()

nside = 32
mapscan, mapcount = healpix_map(ll[scantype != 0], bb[scantype != 0], mytod[scantype != 0], nside=nside, countcut=counts_thr)
mapscan_pos, _ = healpix_map(ll[scantype > 0], bb[scantype > 0], mytod[scantype > 0], nside=nside, countcut=counts_thr)
mapscan_neg, _ = healpix_map(ll[scantype < 0], bb[scantype < 0], mytod[scantype < 0], nside=nside, countcut=counts_thr)

# Display Results
display_all(mapscan, mapscan_pos, mapscan_neg, anatype=anatype, rot=[np.mean(ll), np.mean(bb)], reso=15, TESNum=TESNum)


In [ ]:
# What does PySM say ?
QPindex = iQP2iQS(TESNum-1)
simtod = TOD_sim[QPindex,:]*1e19


mysimtod = mypipe(simtod, tt, azt, elt, scantype, doplot=False, verbose=True,
                  nsplines_global=nsplines_global,
                  lowcut=lowcut,
                  highcut=highcut,
                  nbspl_scan=nbspl_scan,
                  nbins_scan=nbins_scan,
                  order=order, offsets=offsets)

psfake, fffake = ft.power_spectrum(tt, simtod)
psfake2, fffake2 = ft.power_spectrum(tt, mysimtod)

figure()
plot(fffake, psfake, label='Initial', alpha=0.5)
plot(fffake2, psfake2, label='Final', alpha=0.5)
axvline(x=fsignal, ls=':', label='Expected signal location')
axvline(x=lowcut, ls='--', lw=2, color='g', label='High Pass')
axvline(x=highcut, ls=':', lw=2, color='g', label='Low Pass')
xscale('log')
yscale('log')
xlabel('Frequency [Hz]')
ylabel('PSD')
title('PySM through QUBIC')
legend()


figure()
mapscan_sim, mapcount_sim = healpix_map(ll[scantype != 0], bb[scantype != 0], mysimtod[scantype != 0], nside=nside)


maptoplot = mapscan.copy()
mm, ss = ft.meancut(maptoplot[maptoplot != hp.UNSEEN], 3)
hp.gnomview(maptoplot, rot=[np.mean(ll), np.mean(bb)], reso=15, title='QUBIC TES{}'.format(TESNum), sub=(2,3,1), min=-nsig*ss, max=+nsig*ss)
hp.graticule()

mm, ss = ft.meancut(mapscan_sim[maptoplot != hp.UNSEEN], 3)
hp.gnomview(mapscan_sim, rot=[np.mean(ll), np.mean(bb)], reso=15, title='PySM Filtered', sub=(2,3,2), min=-nsig*ss, max=+nsig*ss)
hp.graticule()

fakesky = hp.ud_grade(fakesky, nside)
fakesky[maptoplot == hp.UNSEEN] = hp.UNSEEN
mm, ss = ft.meancut(fakesky[maptoplot != hp.UNSEEN], 3)
hp.gnomview(fakesky, rot=[np.mean(ll), np.mean(bb)], reso=15, title='PySM Raw', sub=(2,3,3), min=-nsig*ss, max=+nsig*ss)
hp.graticule()


In [ ]:
dt = tt[1]-tt[0]
mapexposure = mapcount.copy()
mapexposure[mapcount != hp.UNSEEN] *= dt

hp.gnomview(mapcount, rot=[np.mean(ll), np.mean(bb)], reso=15, sub=(2,3,1), title='Counts')
hp.graticule()

hp.gnomview(mapexposure, rot=[np.mean(ll), np.mean(bb)], reso=15, sub=(2,3,2), title='Exposure [s]')
hp.graticule()



# Check some TES

In [ ]:
reload(tdt)
# Map-making
anatype = ''

#allTES = [33, 93, 94, 95, 96, 133]
allTES = [96, 59, 34, 63, 26, 86]

allmaps = np.zeros((len(allTES), 12*nside**2))
allsig = np.zeros(len(allTES))

for i, TESNum in enumerate(allTES):
    tod = alltod[TESNum-1,:]
    mytod = mypipe(tod, tt, azt, elt, scantype, doplot=False, 
                   nsplines_global=nsplines_global,
                   highcut=highcut,
                   lowcut=lowcut,
                   nbspl_scan=nbspl_scan,
                   nbins_scan=nbins_scan,
                   order=order, offsets=offsets)
    mapscan, mapcount = healpix_map(ll[scantype != 0], bb[scantype != 0], mytod[scantype != 0], nside=nside, countcut=counts_thr)

    # Display Results
    ok = mapscan != hp.UNSEEN
    mm, ss = ft.meancut(mapscan[ok], 3)
    mapscan[ok] -= mm
    allsig[i] = ss
    allmaps[i,:] = mapscan
    hp.gnomview(mapscan, rot=[np.mean(ll), np.mean(bb)], reso=15, min=-nsig*ss, max=nsig*ss, title='TES #{}'.format(TESNum), sub=(2,3,i+1))
    hp.graticule()


In [ ]:
avmap = np.zeros(12*nside**2) + hp.UNSEEN
okpix = allmaps[0,:] != hp.UNSEEN
pixnums = np.arange(12*nside**2)

for i in pixnums[okpix]:
    avmap[i] = ft.weighted_mean(allmaps[:,i], allsig)[0]
    
maptoplot = avmap.copy()
mm, ss = ft.meancut(maptoplot[maptoplot != hp.UNSEEN], 3)
hp.gnomview(maptoplot, rot=[np.mean(ll), np.mean(bb)], reso=15, title='QUBIC 6 TES', sub=(2,3,1), min=-nsig*ss, max=+nsig*ss)
hp.graticule()

mm, ss = ft.meancut(mapscan_sim[maptoplot != hp.UNSEEN], 3)
hp.gnomview(mapscan_sim, rot=[np.mean(ll), np.mean(bb)], reso=15, title='PySM Filtered', sub=(2,3,2), min=-nsig*ss, max=+nsig*ss)
hp.graticule()


# All TES

In [ ]:
reload(tdt)
# Map-making
anatype = ''

allTES = np.arange(256)+1
#nside = 32

allmaps = np.zeros((len(allTES), 12*nside**2))
allsig = np.zeros(len(allTES))
allmean = np.zeros(len(allTES))
for i, TESNum in enumerate(allTES):
    tod = alltod[TESNum-1,:]
    mytod = mypipe(tod, tt, azt, elt, scantype, doplot=False, 
                   nsplines_global=nsplines_global,
                   highcut=highcut,
                   lowcut=lowcut,
                   nbspl_scan=nbspl_scan,
                   nbins_scan=nbins_scan,
                   order=order, offsets=offsets)
    mapscan, mapcount = healpix_map(ll[scantype != 0], bb[scantype != 0], mytod[scantype != 0], nside=nside)

    if i % 4 == 0:
        show()
        figure()
    # Display Results
    ok = mapscan != hp.UNSEEN
    mm, ss = ft.meancut(mapscan[ok], 3)
    mapscan[ok] -= mm
    allmean[i] = mm
    allsig[i] = ss
    allmaps[i,:] = mapscan
    nsig = 3
    hp.gnomview(mapscan, rot=[np.mean(ll), np.mean(bb)], reso=15, min=-nsig*ss, max=nsig*ss, title='TES #{}'.format(TESNum), sub=(1,4,(i % 4)+1))
    hp.graticule()


In [ ]:
### Select the not too crazy one
# hist(np.log10(allsig), range=[-1, 6], bins=50)
# good = (np.log10(allsig) > 2) & (np.log10(allsig) < 4.3)
# hist(np.log10(allsig[good]), range=[-1, 6], bins=50, alpha=0.5)

### Select the not too crazy one
subplot(2,2,1)
hist(np.log10(allsig), range=[np.min(np.log10(allsig)), np.max(np.log10(allsig))], bins=50)
good = (np.log10(allsig) > 2) & (np.log10(allsig) < 4.3)
hist(np.log10(allsig[good]), range=[np.min(np.log10(allsig)), np.max(np.log10(allsig))], bins=50, alpha=0.5)
xlabel('Log10(RMS Map)')


In [ ]:
avmap = np.zeros(12*nside**2) + hp.UNSEEN
medmap = np.zeros(12*nside**2) + hp.UNSEEN
okpix = allmaps[0,:] != hp.UNSEEN
pixnums = np.arange(12*nside**2)

for i in pixnums[okpix]:
    avmap[i] = ft.weighted_mean(allmaps[good,i], allsig[good])[0]
    medmap[i] = np.median(allmaps[good,i])
    
    
mm, ss = ft.meancut(avmap[ok], 3)
hp.gnomview(avmap, rot=[np.mean(ll), np.mean(bb)], reso=15, min=-nsig*ss, max=nsig*ss, title='Average {} TES'.format(good.sum()), sub=[1,3,1])
hp.graticule()

mm, ss = ft.meancut(medmap[ok], 3)
hp.gnomview(medmap, rot=[np.mean(ll), np.mean(bb)], reso=15, min=-nsig*ss, max=nsig*ss, title='Median {} TES'.format(good.sum()), sub=[1,3,2])
hp.graticule()

mm, ss = ft.meancut(mapscan_sim[maptoplot != hp.UNSEEN], 3)
hp.gnomview(mapscan_sim, rot=[np.mean(ll), np.mean(bb)], reso=15, title='PySM Filtered', sub=(1,3,3), min=-nsig*ss, max=+nsig*ss)
hp.graticule()


In [ ]:
### Select the not too crazy one
hist(np.log10(allsig), range=[-1, 6], bins=50)
good = (np.log10(allsig) > 2) & (np.log10(allsig) < 4.3)
hist(np.log10(allsig[good]), range=[-1, 6], bins=50, alpha=0.5)


pixnums = np.arange(12*nside**2)
mymedmap = np.zeros(12*nside**2)
allcalib = np.zeros(256)+1
alloffsets = np.zeros(256)+0
allchi2 = np.zeros(256)
oktes = good.copy()
tesnums = np.arange(256)

niter = 20
n_in = np.zeros(niter)
m_in = np.zeros(niter)
s_in = np.zeros(niter)

### Intital Median
for ipix in pixnums[okpix]:
    mymedmap[ipix] = np.median(allmaps[oktes,ipix])
    
### Iterations
for jiter in range(niter):
    for ites in tesnums[oktes]:
        xdata = mymedmap[okpix]
        ydata = allmaps[ites,okpix]
        errdata = allsig[ites]+np.zeros(okpix.sum())

        allcalib[ites], alloffsets[ites] = np.polyfit(xdata, ydata, 1, w=1./errdata**2)
        residuals = (ydata - (allcalib[ites]*xdata + alloffsets[ites]))/errdata
        allchi2[ites] = np.sum(residuals**2)

    mm, ss = ft.meancut(allchi2, 3)
    ncut = 2.
    oktes = (allchi2 > (mm-ncut*ss)) & (allchi2 < (mm+ncut*ss))
    
    ### New median
    for ipix in pixnums[okpix]:
        mymedmap[ipix] = np.median(allmaps[oktes,ipix] * allcalib[oktes] + alloffsets[oktes])

    n_in[jiter] = oktes.sum()
    m_in[jiter] = np.mean(mymedmap[okpix])
    s_in[jiter] = np.std(mymedmap[okpix])
    print('Iteration {}: Nok={}   Medmap: {} +/- {}'.format(jiter, oktes.sum(), np.mean(mymedmap[okpix]), np.std(mymedmap[okpix])))


subplot(1,3,1)
plot(n_in)
subplot(1,3,2)
plot(m_in)
subplot(1,3,3)
plot(s_in)

fig=figure()
res=25
mymedmap[~okpix] = hp.UNSEEN
mm, ss = ft.meancut(mymedmap[ok], 3)
hp.gnomview(mymedmap, rot=[np.mean(ll), np.mean(bb)], reso=res, min=-nsig*ss, max=nsig*ss, title='Median {} TES'.format(oktes.sum()), sub=[2,4,1])
hp.graticule()

mms, sss = ft.meancut(mapscan_sim[maptoplot != hp.UNSEEN], 3)
hp.gnomview(mapscan_sim, rot=[np.mean(ll), np.mean(bb)], reso=res, title='PySM 150 GHz', sub=[2,4,2], min=-nsig*sss, max=+nsig*sss)
hp.graticule()

def coeff_fit(x, pars):
    return pars[0]*x + pars[1]


#figure()
subplot(2,2,2)
plot(mapscan_sim[okpix], mymedmap[okpix], 'k.')
xxx, yyy, dx, dy, _ = ft.profile(mapscan_sim[okpix], mymedmap[okpix], nbins=10, median=True, rng=[-nsig*sss, nsig*sss], dispersion=False)
errorbar(xxx, yyy, yerr=dy, fmt='ro')
ff = fit.Data(xxx, yyy, dy, coeff_fit)
res = ff.fit_minuit([1., 0])
coeff = res[0].values[0]
errcoeff = res[0].errors[0]
plot(mapscan_sim[okpix], coeff * mapscan_sim[okpix], label='Coeff = {0:5.3f} +/- {1:5.3f}  ({2:4.2f}$\sigma$)'.format(coeff, errcoeff, coeff/errcoeff))
legend()
xlim(mms-6*sss, mms+6*sss)
ylim(mm-6*ss, mm+6*ss)
xlabel('PySM through QUBIC')
ylabel('Median maps with {} TES'.format(oktes.sum()))
title(file)
tight_layout()



ax = fig.add_subplot(2,2,3,projection='mollweide')
grid()
title('Galactic')

phi = np.radians(((qs.galactic[:,0]+180) % 360)-180)
theta = np.radians(qs.galactic[:,1])

plot(phi, theta ,'ro')
xlabel('RA from QS (deg)')
ylabel('DEC from QS (deg)')
tight_layout()


[lowcut, highcut, coeff/errcoeff]

In [ ]:
### Done on file #3  16.24.20
vals = np.array([
        [0.003, 0.1, 1.947481077392674],
        [0.0035, 0.1, 1.2186627708657343],
        [0.0037, 0.1, 1.9234083690839046],
        [0.0038, 0.1, 3.9751534577855234],
        [0.00385, 0.1, 3.7867200415148496],
        [0.0039, 0.1, 3.928096609593341],
        [0.004, 0.1, 2.5417357432779975],
        [0.0045, 0.1, 0.025557650673950844],
        [0.005, 0.1, 0.5351185312405917]
       ])

subplot(2,2,1)
plot(vals[:,0], vals[:,2])
xlabel('Low cut [Hz]')
ylabel('Slope significance (#σ)')


vals = np.array([
    [0.0038, 0.05, 1.1683219828254923],
    [0.0038, 0.08, 2.5293827009127274],
    [0.0038, 0.09, 2.925867960406169],
    [0.0038, 0.095, 2.5391268030652085],
    [0.0038, 0.1, 3.9751534577855234],
    [0.0038, 0.102, 3.580791387669208],
    [0.0038, 0.105, 3.9814124579219317],
    [0.0038, 0.11, 3.897865764760337],
    [0.0038, 0.12, 3.188545525930687],
    [0.0038, 0.15, 2.832994591273594]
       ])

subplot(2,2,3)
plot(vals[:,1], vals[:,2])
xlabel('High cut [Hz]')
ylabel('Slope significance (#σ)')
tight_layout()

In [ ]:
# import pickle

# mystuff = [allmaps, mapscan_sim, mymedmap]

# with open(file+'.pickle', 'wb') as f:
#     pickle.dump(mystuff, f)
